In [1]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import DataLoader

model_name = "bigscience/bloom-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BloomForSequenceClassification(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-23): 24 x BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine

In [2]:
train_jsonl = "alphanli-train-dev/train.jsonl"
train_labels = "alphanli-train-dev/train-labels.lst"
dev_jsonl = "alphanli-train-dev/dev.jsonl"
dev_labels = "alphanli-train-dev/dev-labels.lst"

def load_data(jsonl_file, labels_file):
    data = []
    with open(jsonl_file, "r") as f_json, open(labels_file, "r") as f_labels:
        labels = [int(line.strip()) for line in f_labels.readlines()]
        for idx, line in enumerate(f_json):
            entry = json.loads(line.strip())
            data.append({
                "obs1": entry["obs1"],
                "obs2": entry["obs2"],
                "hyp1": entry["hyp1"],
                "hyp2": entry["hyp2"],
                "label": labels[idx]
            })
    return data

train_data = load_data(train_jsonl, train_labels)
dev_data = load_data(dev_jsonl, dev_labels)

class ANLIDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        entry = self.data[idx]
        premise = f"{entry['obs1']} {entry['obs2']}"

        encoding1 = self.tokenizer(
            premise, entry['hyp1'],
            truncation=True, padding='max_length', max_length=self.max_length,
            return_tensors="pt"
        )
        encoding2 = self.tokenizer(
            premise, entry['hyp2'],
            truncation=True, padding='max_length', max_length=self.max_length,
            return_tensors="pt"
        )

        encoding1 = {k: v.squeeze(0) for k, v in encoding1.items()}
        encoding2 = {k: v.squeeze(0) for k, v in encoding2.items()}

        label = entry["label"] - 1
        return {"input1": encoding1, "input2": encoding2, "label": label}

def collate_fn(batch):
    batch_input1 = {}
    batch_input2 = {}
    keys = list(batch[0]["input1"].keys())
    for k in keys:
        batch_input1[k] = torch.stack([item["input1"][k] for item in batch])
        batch_input2[k] = torch.stack([item["input2"][k] for item in batch])
    labels = torch.tensor([item["label"] for item in batch])
    return {"input1": batch_input1, "input2": batch_input2, "labels": labels}

train_dataset = ANLIDataset(train_data, tokenizer)
dev_dataset = ANLIDataset(dev_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
dev_loader = DataLoader(dev_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

optimizer = AdamW(model.parameters(), lr=2e-5)
num_epochs = 5
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
loss_fn = torch.nn.CrossEntropyLoss()


scaler = torch.cuda.amp.GradScaler()

model.train()
for epoch in range(num_epochs):
    total_loss = 0.0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        optimizer.zero_grad()
        
        batch_input1 = {k: v.to(device) for k, v in batch["input1"].items()}
        batch_input2 = {k: v.to(device) for k, v in batch["input2"].items()}
        labels = batch["labels"].to(device)
        
        with torch.cuda.amp.autocast():
            outputs1 = model(**batch_input1)
            outputs2 = model(**batch_input2)

            score1 = outputs1.logits[:, 0] 
            score2 = outputs2.logits[:, 0]

            logits = torch.stack([score1, score2], dim=1)

            loss = loss_fn(logits, labels)
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
    
    output_dir = f"bloom-aNLI_epoch_{epoch+1}"
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Saved model checkpoint to {output_dir}")

/opt/conda/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_170/1646551368.py:83: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/tmp/ipykernel_170/1646551368.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training Epoch 1: 100%|██████████| 5302/5302 [23:01<00:00,  3.84it/s]


Epoch 1/5, Loss: 0.2816
Saved model checkpoint to bloom-aNLI_epoch_1


Training Epoch 2: 100%|██████████| 5302/5302 [23:01<00:00,  3.84it/s]


Epoch 2/5, Loss: 0.0879
Saved model checkpoint to bloom-aNLI_epoch_2


Training Epoch 3: 100%|██████████| 5302/5302 [23:01<00:00,  3.84it/s]


Epoch 3/5, Loss: 0.0287
Saved model checkpoint to bloom-aNLI_epoch_3


Training Epoch 4: 100%|██████████| 5302/5302 [23:00<00:00,  3.84it/s]


Epoch 4/5, Loss: 0.0100
Saved model checkpoint to bloom-aNLI_epoch_4


Training Epoch 5: 100%|██████████| 5302/5302 [23:02<00:00,  3.84it/s]


Epoch 5/5, Loss: 0.0020
Saved model checkpoint to bloom-aNLI_epoch_5


In [4]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in tqdm(dev_loader, desc="Evaluating"):

        batch_input1 = {k: v.to(device) for k, v in batch["input1"].items()}
        batch_input2 = {k: v.to(device) for k, v in batch["input2"].items()}
        labels = batch["labels"].to(device)
        
        outputs1 = model(**batch_input1)
        outputs2 = model(**batch_input2)
        
        score1 = outputs1.logits[:, 0]
        score2 = outputs2.logits[:, 0]
        logits = torch.stack([score1, score2], dim=1)
        
        preds = torch.argmax(logits, dim=1) + 1
        
        all_preds.extend(preds.cpu().numpy().tolist())
        all_labels.extend((labels.cpu().numpy() + 1).tolist())

accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(
    all_labels, all_preds, average="binary", pos_label=1
)

print(f"Dev Accuracy: {accuracy:.4f}")
print(f"Dev Precision: {precision:.4f}")
print(f"Dev Recall: {recall:.4f}")
print(f"Dev F1-score: {f1:.4f}")

Evaluating: 100%|██████████| 48/48 [00:17<00:00,  2.74it/s]

Dev Accuracy: 0.6384
Dev Precision: 0.6499
Dev Recall: 0.6300
Dev F1-score: 0.6398
